In [2]:
import pandas as pd

df = pd.read_csv('test_dataset.csv')
df = df.drop('expected_answer', axis=1)
df.to_csv('test_dataset.csv', index=False)

# Variable Refactorization

In [1]:
import pandas as pd
import re
import string

def create_variable_mapping():
    """
    Create mapping from A,B,C,... to Z,Y,X,... as used in CORR2CAUSE paper
    """
    original_vars = list(string.ascii_uppercase)  # A, B, C, ..., Z
    refactored_vars = list(string.ascii_uppercase[::-1])  # Z, Y, X, ..., A
    
    return dict(zip(original_vars, refactored_vars))

def refactor_variables_in_text(text, var_mapping):
    """
    Replace variable names in text according to the mapping
    Uses word boundaries to avoid partial replacements
    Uses temporary placeholders to avoid circular replacements
    """
    refactored_text = text
    
    # First pass: replace original variables with temporary placeholders
    temp_mapping = {}
    for i, (original_var, refactored_var) in enumerate(var_mapping.items()):
        temp_placeholder = f"__TEMP_VAR_{i}__"
        temp_mapping[temp_placeholder] = refactored_var
        
        # Replace original variable with temporary placeholder
        pattern = r'\b' + re.escape(original_var) + r'\b'
        refactored_text = re.sub(pattern, temp_placeholder, refactored_text)
    
    # Second pass: replace temporary placeholders with final variables
    for temp_placeholder, refactored_var in temp_mapping.items():
        refactored_text = refactored_text.replace(temp_placeholder, refactored_var)
    
    return refactored_text

def apply_variable_refactorization(df):
    """
    Apply variable refactorization to dataset following CORR2CAUSE approach
    Swaps A,B,C,... to Z,Y,X,... in the input text
    """
    var_mapping = create_variable_mapping()
    df_refactored = df.copy()
    
    # Apply refactorization to the 'input' column
    df_refactored['input'] = df_refactored['input'].apply(
        lambda x: refactor_variables_in_text(x, var_mapping)
    )
    
    # If there's an 'expected_answer' column with variable names, refactor it too
    if 'expected_answer' in df_refactored.columns:
        df_refactored['expected_answer'] = df_refactored['expected_answer'].apply(
            lambda x: refactor_variables_in_text(str(x), var_mapping) if pd.notna(x) else x
        )
    
    return df_refactored

# Load the test dataset
print("Loading test dataset...")
df = pd.read_csv('test_dataset.csv')
print(f"Original dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

Loading test dataset...
Original dataset shape: (1162, 4)
Columns: ['input', 'label', 'num_variables', 'template']


In [5]:
# Example usage and testing
print("Testing variable refactorization...")

# Show the variable mapping
var_mapping = create_variable_mapping()
print(f"Variable mapping (first 10): {dict(list(var_mapping.items())[:10])}")

# Test on a sample from the dataset
sample_text = df['input'].iloc[0]
print("\nOriginal text sample:")
print(sample_text[:200] + "...")

# Debug: let's see what variables are actually in the text
variables_in_text = re.findall(r'\b[A-Z]\b', sample_text)
print(f"\nVariables found in original text: {variables_in_text}")

refactored_text = refactor_variables_in_text(sample_text, var_mapping)
print("\nRefactored text sample:")
print(refactored_text[:200] + "...")

# Debug: check variables in refactored text
variables_in_refactored = re.findall(r'\b[A-Z]\b', refactored_text)
print(f"Variables found in refactored text: {variables_in_refactored}")

# Show the actual difference
if sample_text != refactored_text:
    print("\n✓ Refactorization worked!")
else:
    print("\n✗ Refactorization failed - texts are identical")

Testing variable refactorization...
Variable mapping (first 10): {'A': 'Z', 'B': 'Y', 'C': 'X', 'D': 'W', 'E': 'V', 'F': 'U', 'G': 'T', 'H': 'S', 'I': 'R', 'J': 'Q'}

Original text sample:
Premise: Suppose there is a closed system of 2 variables, A and B. All the statistical relations among these 2 variables are as follows: A correlates with B.
Hypothesis: There exists at least one coll...

Variables found in original text: ['A', 'B', 'A', 'B', 'A', 'B']

Refactored text sample:
Premise: Suppose there is a closed system of 2 variables, Z and Y. All the statistical relations among these 2 variables are as follows: Z correlates with Y.
Hypothesis: There exists at least one coll...
Variables found in refactored text: ['Z', 'Y', 'Z', 'Y', 'Z', 'Y']

✓ Refactorization worked!


In [4]:
# Apply refactorization to the entire dataset
print("Applying variable refactorization to entire dataset...")
df_refactored = apply_variable_refactorization(df)

print(f"Refactored dataset shape: {df_refactored.shape}")

# Compare original vs refactored samples
print("\n" + "="*50)
print("COMPARISON: Original vs Refactored")
print("="*50)

for i in range(min(3, len(df))):
    print(f"\nSample {i+1}:")
    print(f"Original: {df['input'].iloc[i][:150]}...")
    print(f"Refactored: {df_refactored['input'].iloc[i][:150]}...")
    print("-" * 30)

Applying variable refactorization to entire dataset...
Refactored dataset shape: (1162, 4)

COMPARISON: Original vs Refactored

Sample 1:
Original: Premise: Suppose there is a closed system of 2 variables, A and B. All the statistical relations among these 2 variables are as follows: A correlates ...
Refactored: Premise: Suppose there is a closed system of 2 variables, Z and Y. All the statistical relations among these 2 variables are as follows: Z correlates ...
------------------------------

Sample 2:
Original: Premise: Suppose there is a closed system of 2 variables, A and B. All the statistical relations among these 2 variables are as follows: A correlates ...
Refactored: Premise: Suppose there is a closed system of 2 variables, Z and Y. All the statistical relations among these 2 variables are as follows: Z correlates ...
------------------------------

Sample 3:
Original: Premise: Suppose there is a closed system of 2 variables, A and B. All the statistical relations among these

In [6]:
# Save the refactored dataset
output_filename = 'test_dataset_variable_refactorization.csv'
df_refactored.to_csv(output_filename, index=False)
print(f"\nRefactored dataset saved as: {output_filename}")

# Verify the transformation worked correctly
print("\nVerification:")
original_vars = sorted(set(re.findall(r'\b[A-Z]\b', ' '.join(df['input'].head(10)))))
refactored_vars = sorted(set(re.findall(r'\b[A-Z]\b', ' '.join(df_refactored['input'].head(10)))))
print(f"Original dataset has variables like: {original_vars}")
print(f"Refactored dataset has variables like: {refactored_vars}")

print("\nVariable refactorization complete! This follows the CORR2CAUSE approach where:")
print("- Original variables A, B, C, ... are mapped to Z, Y, X, ...")
print("- This tests model robustness by changing variable names while preserving relationships")


Refactored dataset saved as: test_dataset_variable_refactorization.csv

Verification:
Original dataset has variables like: ['A', 'B', 'C']
Refactored dataset has variables like: ['X', 'Y', 'Z']

Variable refactorization complete! This follows the CORR2CAUSE approach where:
- Original variables A, B, C, ... are mapped to Z, Y, X, ...
- This tests model robustness by changing variable names while preserving relationships


# Hypothesis Paraphrasing

In [7]:
# Hypothesis Paraphrasing Implementation
# Based on CORR2CAUSE paper Appendix C

def create_paraphrasing_templates():
    """
    Create mapping between original templates and their paraphrases
    Based on CORR2CAUSE paper Table 9
    """
    # Map dataset template names to CORR2CAUSE naming
    template_mapping = {
        'parent': 'Is-Parent',
        'non-parent ancestor': 'Is-Ancestor', 
        'child': 'Is-Child',
        'non-child descendant': 'Is-Descendant',
        'has_collider': 'Has-Collider',
        'has_confounder': 'Has-Confounder'
    }
    
    # Original templates from CORR2CAUSE paper
    original_templates = {
        'Is-Parent': '{Var i} directly causes {Var j}.',
        'Is-Ancestor': '{Var i} causes something else which causes {Var j}.',
        'Is-Child': '{Var j} directly causes {Var i}.',
        'Is-Descendant': '{Var j} is a cause for {Var i}, but not a direct one.',
        'Has-Collider': 'There exists at least one collider (i.e., common effect) of {Var i} and {Var j}.',
        'Has-Confounder': 'There exists at least one confounder (i.e., common cause) of {Var i} and {Var j}.'
    }
    
    # Paraphrased templates from CORR2CAUSE paper  
    paraphrased_templates = {
        'Is-Parent': '{Var i} directly affects {Var j}.',
        'Is-Ancestor': '{Var i} influences {Var j} through some mediator(s).',
        'Is-Child': '{Var j} directly affects {Var i}.',
        'Is-Descendant': '{Var j} influences {Var i} through some mediator(s).',
        'Has-Collider': '{Var i} and {Var j} together cause some other variable(s).',
        'Has-Confounder': 'Some variable(s) cause(s) both {Var i} and {Var j}.'
    }
    
    return template_mapping, original_templates, paraphrased_templates

def extract_variables_from_hypothesis(hypothesis_text):
    """
    Extract variable names from hypothesis text
    Returns the first two single-letter variables found
    """
    variables = re.findall(r'\b[A-Z]\b', hypothesis_text)
    if len(variables) >= 2:
        return variables[0], variables[1]
    return None, None

def paraphrase_hypothesis(text, template_name, template_mapping, original_templates, paraphrased_templates):
    """
    Replace the hypothesis in text with its paraphrased version
    """
    if template_name not in template_mapping:
        return text  # Return unchanged if template not recognized
    
    # Split premise and hypothesis
    if 'Hypothesis: ' not in text:
        return text
    
    premise, hypothesis = text.split('Hypothesis: ', 1)
    
    # Extract variables from the hypothesis
    var_i, var_j = extract_variables_from_hypothesis(hypothesis)
    if var_i is None or var_j is None:
        return text  # Return unchanged if variables not found
    
    # Get the paraphrased template
    corr2cause_template = template_mapping[template_name]
    paraphrased_template = paraphrased_templates[corr2cause_template]
    
    # Replace placeholder variables with actual variables
    paraphrased_hypothesis = paraphrased_template.replace('{Var i}', var_i).replace('{Var j}', var_j)
    
    # Reconstruct the full text
    paraphrased_text = premise + 'Hypothesis: ' + paraphrased_hypothesis
    
    return paraphrased_text

def apply_hypothesis_paraphrasing(df):
    """
    Apply hypothesis paraphrasing to the entire dataset
    """
    template_mapping, original_templates, paraphrased_templates = create_paraphrasing_templates()
    
    df_paraphrased = df.copy()
    
    # Apply paraphrasing to each row
    df_paraphrased['input'] = df_paraphrased.apply(
        lambda row: paraphrase_hypothesis(
            row['input'], 
            row['template'], 
            template_mapping, 
            original_templates, 
            paraphrased_templates
        ), 
        axis=1
    )
    
    return df_paraphrased

print("Hypothesis paraphrasing functions defined!")

Hypothesis paraphrasing functions defined!


In [8]:
# Test hypothesis paraphrasing
print("Testing hypothesis paraphrasing...")

# Test on one example from each template
template_mapping, original_templates, paraphrased_templates = create_paraphrasing_templates()

print("Template mappings:")
for dataset_template, corr2cause_template in template_mapping.items():
    print(f"  {dataset_template} -> {corr2cause_template}")

print("\n" + "="*60)
print("PARAPHRASING EXAMPLES")
print("="*60)

for template in df['template'].unique():
    # Get a sample from this template
    sample_row = df[df['template'] == template].iloc[0]
    original_text = sample_row['input']
    
    # Apply paraphrasing
    paraphrased_text = paraphrase_hypothesis(
        original_text, 
        template, 
        template_mapping, 
        original_templates, 
        paraphrased_templates
    )
    
    # Extract just the hypothesis parts for comparison
    original_hyp = original_text.split('Hypothesis: ')[1] if 'Hypothesis: ' in original_text else 'N/A'
    paraphrased_hyp = paraphrased_text.split('Hypothesis: ')[1] if 'Hypothesis: ' in paraphrased_text else 'N/A'
    
    print(f"\nTemplate: {template}")
    print(f"Original:    {original_hyp}")
    print(f"Paraphrased: {paraphrased_hyp}")
    print("-" * 40)

Testing hypothesis paraphrasing...
Template mappings:
  parent -> Is-Parent
  non-parent ancestor -> Is-Ancestor
  child -> Is-Child
  non-child descendant -> Is-Descendant
  has_collider -> Has-Collider
  has_confounder -> Has-Confounder

PARAPHRASING EXAMPLES

Template: has_collider
Original:    There exists at least one collider (i.e., common effect) of A and B.
Paraphrased: A and B together cause some other variable(s).
----------------------------------------

Template: parent
Original:    A directly causes B.
Paraphrased: A directly affects B.
----------------------------------------

Template: non-child descendant
Original:    A is a cause for B, but not a direct one.
Paraphrased: B influences A through some mediator(s).
----------------------------------------

Template: child
Original:    B directly causes A.
Paraphrased: A directly affects B.
----------------------------------------

Template: non-parent ancestor
Original:    B causes something else which causes A.
Paraphrase

In [9]:
# Apply hypothesis paraphrasing to the entire dataset
print("Applying hypothesis paraphrasing to entire dataset...")
df_paraphrased = apply_hypothesis_paraphrasing(df)

print(f"Paraphrased dataset shape: {df_paraphrased.shape}")

# Save the paraphrased dataset
output_filename = 'test_dataset_hypothesis_paraphrased.csv'
df_paraphrased.to_csv(output_filename, index=False)
print(f"Paraphrased dataset saved as: {output_filename}")

# Show some examples of the paraphrasing
print("\n" + "="*60)
print("COMPARISON: Original vs Paraphrased Hypotheses")
print("="*60)

for template in df['template'].unique()[:3]:  # Show first 3 templates
    original_sample = df[df['template'] == template].iloc[0]
    paraphrased_sample = df_paraphrased[df_paraphrased['template'] == template].iloc[0]
    
    orig_hyp = original_sample['input'].split('Hypothesis: ')[1] if 'Hypothesis: ' in original_sample['input'] else 'N/A'
    para_hyp = paraphrased_sample['input'].split('Hypothesis: ')[1] if 'Hypothesis: ' in paraphrased_sample['input'] else 'N/A'
    
    print(f"\nTemplate: {template}")
    print(f"Original:    {orig_hyp}")
    print(f"Paraphrased: {para_hyp}")
    print("-" * 40)

print("\nHypothesis paraphrasing complete! This follows the CORR2CAUSE approach where:")
print("- Original hypothesis templates are replaced with semantically equivalent paraphrases")
print("- This tests if models truly learned reasoning rather than overfitting to surface phrasing")
print("- The paraphrased dataset can reveal model dependency on specific wording patterns")

Applying hypothesis paraphrasing to entire dataset...
Paraphrased dataset shape: (1162, 4)
Paraphrased dataset saved as: test_dataset_hypothesis_paraphrased.csv

COMPARISON: Original vs Paraphrased Hypotheses

Template: has_collider
Original:    There exists at least one collider (i.e., common effect) of A and B.
Paraphrased: A and B together cause some other variable(s).
----------------------------------------

Template: parent
Original:    A directly causes B.
Paraphrased: A directly affects B.
----------------------------------------

Template: non-child descendant
Original:    A is a cause for B, but not a direct one.
Paraphrased: B influences A through some mediator(s).
----------------------------------------

Hypothesis paraphrasing complete! This follows the CORR2CAUSE approach where:
- Original hypothesis templates are replaced with semantically equivalent paraphrases
- This tests if models truly learned reasoning rather than overfitting to surface phrasing
- The paraphrased 